In [2]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from skimage.filters import threshold_otsu
import seaborn as sns
from skimage.color import rgb2gray
from skimage.morphology import convex_hull_image
from scipy.spatial import ConvexHull
from skimage.measure import label, regionprops
from skimage.draw import polygon,rectangle,polygon_perimeter
import math
import csv
import os

In [3]:
path=r"data_1\0.jpg"
cropped_path=r"arbitrary\x.jpg"

In [6]:
def Feature_1(image_path):
    # Load the image and convert to grayscale
    image = Image.open(image_path).convert("L")
    image_array = np.array(image)
    
    # Calculate threshold using Otsu's method
    dynamic_threshold = threshold_otsu(image_array)
   # print(dynamic_threshold)
    
    # Cap the threshold at a minimum of 200
    threshold = np.mean([dynamic_threshold, 200])

    # Compute the histogram of pixel intensities
    unique, counts = np.unique(image_array, return_counts=True)
    intensity_counts = dict(zip(unique, counts))

    # Identify values where the frequency is greater than 1
    valid_values = {val for val, freq in intensity_counts.items() if freq > 1}

    # Count pixels that are both above the threshold and meet the frequency condition
    high_intensity_count = np.sum([pixel in valid_values and pixel >= threshold for pixel in image_array.flatten()])
    
    # Normalize the count
    normalized_count = high_intensity_count / image_array.size
    # print(normalized_count)
    
    # # Plot histogram
    # plt.figure(figsize=(8, 5))
    # sns.histplot(image_array.flatten(), bins=256, kde=True, color='blue', alpha=0.7)
    # plt.axvline(threshold, color='red', linestyle='--', label=f'Threshold: {threshold:.2f}')
    # plt.title('Pixel Intensity Distribution')
    # plt.xlabel('Pixel Intensity')
    # plt.ylabel('Frequency')
    # plt.legend()
    # plt.show()
    
    return float(f"{normalized_count:.4f}")
Feature_1(path)

0.184

In [7]:
#Frobius Norm on entire Image

def Feature_2(image_path):
    # Load the image and convert it to grayscale
    image = Image.open(image_path).convert("L")
    
    # Convert the image to a numpy array
    image_array = np.array(image)
    
    # Compute the Frobenius norm
    frobenius_norm_value = np.linalg.norm(image_array, 'fro')
    
    return float(f"{frobenius_norm_value:.4f}")

Feature_2(path)

106401.598

In [8]:
def draw_line(r1, c1, r2, c2):
    """
    Generate pixel coordinates for a straight line between two points.
    """
    from skimage.draw import line
    return line(int(r1), int(c1), int(r2), int(c2))

In [9]:

def crop(image_path):
    """
    Applies convex hull processing to the input image, crops the region inside the convex hull,
    shows the overlay image and cropped image side by side in subplots, and returns the cropped image.
    
    Parameters:
        image_path (str): Path to the image file.
        
    Returns:
        np.ndarray: Cropped image inside the convex hull.
    """
    cropped_path=r"arbitrary\x.jpg"
    
    # Load the image
    image = np.array(Image.open(image_path))
    if image.ndim == 3:  # If it's an RGB image
        gray_image = rgb2gray(image)  # Convert to grayscale
    else:
        gray_image = image  # If already grayscale
    
    # Binarize the image using Otsu's threshold
    threshold = threshold_otsu(gray_image)
    binary_image = gray_image >= threshold  # Convert to binary (foreground/background)
    
    # Label connected components (foreground objects)
    labeled_image, num_labels = label(binary_image, connectivity=2, return_num=True)
    
    # Calculate properties of the labeled regions (connected components)
    regions = regionprops(labeled_image)
    
    # Find the region with the largest area (most significant region)
    largest_region = max(regions, key=lambda r: r.area)
    
    # Get the coordinates of the largest region
    largest_region_coords = largest_region.coords
    
    # Compute the convex hull of the largest region's coordinates
    hull = ConvexHull(largest_region_coords)
    
    # Get the vertices of the convex hull
    hull_points = largest_region_coords[hull.vertices]
    
    # Create a mask from the convex hull
    mask = np.zeros_like(binary_image, dtype=bool)
    rr, cc = polygon(hull_points[:, 0], hull_points[:, 1], mask.shape)
    mask[rr, cc] = True
    
     # Create a bounding box (4-cornered box) around the convex hull
    min_row, min_col = hull_points.min(axis=0)
    max_row, max_col = hull_points.max(axis=0)
    box_corners = np.array([[min_row, min_col],
                            [min_row, max_col],
                            [max_row, max_col],
                            [max_row, min_col]])
    
    # Overlay the green box on the original image
    overlay_image = image.copy()
    if image.ndim == 2:  # Convert grayscale to RGB for visualization
        overlay_image = np.stack([overlay_image] * 3, axis=-1)
    
    # Draw the box
    for i in range(len(box_corners)):
        start = box_corners[i]
        end = box_corners[(i + 1) % len(box_corners)]  # Next corner (wrap around)
        rr, cc = draw_line(start[0], start[1], end[0], end[1])
        overlay_image[rr, cc] = [0, 255, 0]  # Green color for the box
    # Overlay the green box on the original image
    overlay_image = image.copy()
    if image.ndim == 2:  # Convert grayscale to RGB for visualization
        overlay_image = np.stack([overlay_image] * 3, axis=-1)
    
    # Draw the box
    for i in range(len(box_corners)):
        start = box_corners[i]
        end = box_corners[(i + 1) % len(box_corners)]  # Next corner (wrap around)
        rr, cc = draw_line(start[0], start[1], end[0], end[1])
        overlay_image[rr, cc] = [0, 255, 0]  # Green color for the box
    
    # Use the mask to extract the region inside the convex hull
    cropped_image = image[int(min_row):int(max_row), int(min_col):int(max_col)]
    
    # Create subplot with 2 images side by side
    # plt.figure(figsize=(12, 6))  # Adjusted figure size for better clarity
    
    # # Plot the overlay image with convex hull rectangle
    # plt.subplot(1, 2, 1)  # (rows, cols, position)
    # plt.imshow(overlay_image)
    # plt.title('Overlay Image with Convex Hull (Rectangle)')
    # plt.axis('off')
    
    # # Plot the cropped image
    # plt.subplot(1, 2, 2)  # (rows, cols, position)
    # plt.imshow(cropped_image)
    # plt.title('Cropped Image Inside Convex Hull')
    # plt.axis('off')
    
    # # Show the subplots
    # plt.tight_layout()
    # plt.show()
    
    # Save the cropped image as a new file
    cropped_image_pil = Image.fromarray(cropped_image)
    cropped_image_pil.save(cropped_path)
    
   

crop(path)

In [10]:
def Feature_3(path):
    crop(path)
    return Feature_2(cropped_path)

Feature_3(path)


103157.1785

In [11]:
def Feature_4():
    path = r"arbitrary\x.jpg"  # Path to your image file
    
    # Open the image and get its dimensions
    image = Image.open(path)
    width, height = image.size  # width and height of the image
    
    # Calculate the diagonal length using Pythagorean theorem
    diagonal_length = math.sqrt(width**2 + height**2)
    
    return float(f"{diagonal_length:.4f}")
Feature_4()
# # Example usage
# diagonal = Feature_4()
# # print(diagonal)

1293.4311

In [12]:
def Feature_5():
    path = r"arbitrary\x.jpg"  # Path to your image file
    
    # Open the image and get its dimensions
    image = Image.open(path)
    width, height = image.size  # width and height of the image
    
    return width

# Example usage
# width = Feature_5()
# print(width)
Feature_5()

592

In [13]:
def Feature_6():
    path = r"arbitrary\x.jpg"  # Path to your image file
    
    # Open the image and get its dimensions
    image = Image.open(path)
    width, height = image.size  # width and height of the image
    
    return height

# Example usage
# height = Feature_6()
# print(height)
Feature_6()

1150

In [14]:
def Feature_7():
    image_path = r"arbitrary\x.jpg"
    # Load the image and convert to grayscale
    image = Image.open(image_path).convert("L")
    image_array = np.array(image)
    
    # Calculate threshold using Otsu's method
    dynamic_threshold = threshold_otsu(image_array)
    # print(dynamic_threshold)
    
    # Cap the threshold at a minimum of 200
    threshold = np.mean([dynamic_threshold, 200])

    # Compute the histogram of pixel intensities
    unique, counts = np.unique(image_array, return_counts=True)
    intensity_counts = dict(zip(unique, counts))

    # Identify values where the frequency is greater than 1
    valid_values = {val for val, freq in intensity_counts.items() if freq > 1}

    # Count pixels that are both above the threshold and meet the frequency condition
    high_intensity_count = np.sum([pixel in valid_values and pixel <= threshold for pixel in image_array.flatten()])
    
    # Normalize the count
    normalized_count = high_intensity_count / image_array.size
    # print(normalized_count)
    
    # # Plot histogram
    # plt.figure(figsize=(8, 5))
    # sns.histplot(image_array.flatten(), bins=256, kde=True, color='blue', alpha=0.7)
    # plt.axvline(threshold, color='red', linestyle='--', label=f'Threshold: {threshold:.2f}')
    # plt.title('Pixel Intensity Distribution')
    # plt.xlabel('Pixel Intensity')
    # plt.ylabel('Frequency')
    # plt.legend()
    # plt.show()
    
    return float(f"{normalized_count:.4f}")
Feature_7()

0.7705

In [15]:
def Feature_8(image_path):
    image = Image.open(image_path).convert("L")
    image = np.array(image)
    # Define the threshold (127 by default)
    threshold = 127

    # Calculate the number of black and white family pixels
    black_family_pixels = np.sum(image <= threshold)
    white_family_pixels = np.sum(image > threshold)
    ratio=0.0
    # Calculate the ratio
    ratio = white_family_pixels / black_family_pixels
    # if white_family_pixels == 0:
    #     print("No white family pixels found. Ratio cannot be calculated.")
    # else:
       
    #     print(f"Ratio of black family to white family pixels: {ratio:.2f}")
    return float(f"{ratio:.4f}")
Feature_8(path)

0.2313

In [16]:
def Feature(path)->list:
    l:list = [Feature_1(path),Feature_2(path),Feature_3(path),Feature_4(),Feature_5(),Feature_6(),Feature_7(),Feature_8(path)]
    return l

Feature(path)

[0.184, 106401.598, 103157.1785, 1293.4311, 592, 1150, 0.7705, 0.2313]

In [17]:
output_folder = './data_2'
os.makedirs(output_folder, exist_ok=True)

# Output CSV file path
output_csv = os.path.join(output_folder, 'dataset.csv')

# Open the CSV file for writing
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write header (optional, adjust as needed)
    csv_writer.writerow(['Feature1', 'Feature2', 'Feature3','Feature4', 'Feature5', 'Feature6','Feature7','Feature8'])  # Update with actual feature names

    # Loop through the range of image files
    for i in range(300):
        # Construct the input file path
        input_path = f'./data_1/{i}.jpg'

        # Check if the file exists
        if os.path.exists(input_path):
            # Get features from the file
            features = Feature(input_path)

            # Write the features to the CSV file
            csv_writer.writerow(features)
        else:
            print(f"File not found: {input_path}")
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27